# Building a simple collaborative filtering recommender system

In this Notebook I will explore the subject of recommender systems. The goal will be to understand how a recommender system works, and after that, to build a simple one.

## 1 Building a simple Recommender system

To start I will build a simple collaborative filtering algorithm that will recommend new items to a user by finding similar items or similar users.

### 1.1 Dataset

To do testing, I decided to use a movie dataset, containing a list of movies, users and ratings.

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History
and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4,
Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872

Let's explore this dataset !

In [1]:
import pandas as pd

ratings_data = pd.read_csv("datasets/movies/ratings.dat", delimiter="::", engine='python', encoding='latin1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

ratings_data.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


This file contains the data about the ratings, the first column is the userid, the second one is the id of the movie, the third one is the rating ($r$) such that $$r \in \{ n \in \mathbb{Z}| 1 \leq r \leq 5 \} $$

We also have access to the timestamp from when the rating was published by the user.

In [2]:
movies_data = pd.read_csv("datasets/movies/movies.dat", delimiter="::", engine='python', encoding='latin1', header=None, names=['MovieID', 'Title', 'Genres'])
movies_data.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


For the movies data, like the ratings data, the format is the same, column are separated using the "::" separator, and the genres for the movies are separated by "|". There is a thirs file, we will, come back to it later.

### 1.2 User-based collaborative filtering

Now that we explored our dataset, the first implementation will be User-based, meaning that we will try to recommend a movie to a user by comparing users ratings on certain movies.

To do that, I will use cosine similarity, and then retrieve the closest neighbors to get similar items.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# first let's create a matrix, the rows will represent each user, and the each column a movie,
# and then the value represents the rating the user gave to that movie (0 = no rating given)

ratings_matrix = ratings_data.pivot_table(index='UserID', columns='MovieID', values='Rating', fill_value=0)

ratings_matrix.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


What we did here is pivot the dataframe to create a 2D array (matrix), where each index/row is the id of each users (they range from 1 to 6040) and each column is a movie id (ranging from 1 to 3952). Each values placed are the rating for each movie and user, the default value in case a user didn't post any review for a film is 0.

In [12]:
# Now we compute the consine similarity between users
from pandas import DataFrame as df

user_similarity = cosine_similarity(ratings_matrix)

df(user_similarity)

,0,1,2,3,4,5,6,7,8,9,...,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039
0,1.000000,0.096382,0.120610,0.132455,0.090158,0.179222,0.059678,0.138241,0.226148,0.255288,...,0.170588,0.082006,0.069807,0.033663,0.114877,0.186329,0.135979,0.000000,0.174604,0.133590
1,0.096382,1.000000,0.151479,0.171176,0.114394,0.100865,0.305787,0.203337,0.190198,0.226861,...,0.112503,0.091222,0.268565,0.014286,0.183384,0.228241,0.206274,0.066118,0.066457,0.218276
2,0.120610,0.151479,1.000000,0.151227,0.062907,0.074603,0.138332,0.077656,0.126457,0.213655,...,0.092960,0.125864,0.161507,0.000000,0.097308,0.143264,0.107744,0.120234,0.094675,0.133144
3,0.132455,0.171176,0.151227,1.000000,0.045094,0.013529,0.130339,0.100856,0.093651,0.120738,...,0.163629,0.093041,0.382803,0.000000,0.082097,0.170583,0.127464,0.062907,0.064634,0.137968
4,0.090158,0.114394,0.062907,0.045094,1.000000,0.047449,0.126257,0.220817,0.261330,0.117052,...,0.100652,0.035732,0.061806,0.054151,0.179083,0.293365,0.172686,0.020459,0.027689,0.241437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.186329,0.228241,0.143264,0.170583,0.293365,0.093583,0.122441,0.227400,0.239607,0.338072,...,0.131294,0.209843,0.186426,0.103431,0.267405,1.000000,0.341462,0.124174,0.219115,0.411891
6036,0.135979,0.206274,0.107744,0.127464,0.172686,0.065788,0.111673,0.144395,0.225055,0.246902,...,0.142309,0.276134,0.129985,0.118749,0.141676,0.341462,1.000000,0.049015,0.252146,0.428240
6037,0.000000,0.066118,0.120234,0.062907,0.020459,0.065711,0.000000,0.019242,0.093470,0.113789,...,0.108837,0.106897,0.040689,0.000000,0.063967,0.124174,0.049015,1.000000,0.161714,0.099300
6038,0.174604,0.066457,0.094675,0.064634,0.027689,0.167303,0.014977,0.044660,0.046434,0.296776,...,0.118776,0.250994,0.053750,0.102168,0.068399,0.219115,0.252146,0.161714,1.000000,0.228332


For each user, we have a score for every other user that represents how similar they are. The closer the cosine value is to 1, the more similar they are.

In [5]:
# for example let's see how similar the user with the id 1 is to itself
user_similarity[1][1]

1.0

Indeed, if we compare the same user, we can see that they are completly identical. Now that we established this matrix, we can compute for each user his closest neighbor(s)

In [13]:
import numpy as np

num_users = user_similarity.shape[0]  # Number of users
N = 5  # Number of nearest neighbors to find, excluding the user itself

nearest_neighbors = []

for user_idx in range(num_users):
    # Get the similarity scores for the current user and sort them in descending order
    sorted_user_similarities = np.argsort(-user_similarity[user_idx])
    
    # Select the top N+1 scores (including the user itself) and exclude the user's own index
    top_user_neighbors = sorted_user_similarities[1:N+1]
    
    # Append the indices (user IDs) of the top N neighbors to the list
    nearest_neighbors.append(top_user_neighbors)

# Convert the list to a numpy array for easier handling
nearest_neighbors = np.array(nearest_neighbors)

# Now, `nearest_neighbors` contains the indices of the N nearest neighbors for each user

df(nearest_neighbors)


,0,1,2,3,4
0,5342,5189,1480,1282,5704
1,3107,94,2813,4600,2302
2,2999,478,5690,3499,1903
3,4142,1574,5875,561,86
4,1483,5451,5748,4606,224
...,...,...,...,...,...
6035,1014,4276,1284,4168,1697
6036,2063,2477,4166,4252,2139
6037,2208,3582,4653,757,2798
6038,930,3024,4853,1321,4929


With the code above, we now know for each user what are their top five similar users.

Now, let's predict how a user might rate an item based on how similar they are with other users

In [155]:
def predict_ratings(ratings, similarity):
    weighted_ratings_sum = similarity @ ratings

    #sum_of_similarities = np.abs(similarity).sum(axis=1)

    #predicted_ratings = weighted_ratings_sum / sum_of_similarities[:, np.newaxis]

    return weighted_ratings_sum


The function above will simply return the matrix multiplication between the similarity of each users and the ratings a user gave for each films.

In [16]:
items_predictions = predict_ratings(ratings_matrix, user_similarity)

items_predictions

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1435.324820,352.206077,203.005986,69.029559,134.841432,473.539984,237.556069,36.138503,32.154485,433.658734,...,39.061123,2.270668,10.360693,25.185841,18.891094,423.164701,169.652108,28.483405,17.689056,189.891745
1,1610.038795,455.268354,277.853937,93.306485,178.981186,815.923891,319.113256,38.090988,65.504694,727.952680,...,50.357122,2.571836,9.647453,41.283574,32.626468,553.388846,228.434877,39.466136,21.568331,269.925455
2,1456.131223,401.590802,231.535374,69.549493,145.946906,621.256560,248.061499,33.212960,49.091738,594.925232,...,41.221632,2.021605,10.637998,32.720535,25.545875,466.714987,182.834751,30.826507,15.451143,198.722552
3,1035.227242,297.364282,151.114699,44.144880,90.678072,530.891895,158.018866,23.241963,39.415405,461.093366,...,29.791334,1.389716,7.039464,23.786485,19.350967,331.860848,139.591794,24.059500,11.581286,149.935977
4,1450.398555,361.406028,227.395750,89.563865,144.952793,731.845441,259.469387,31.665709,43.642760,538.146103,...,59.423340,2.894643,8.557900,36.384605,29.571643,543.921562,266.689173,41.006484,27.902781,276.849143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,2341.659451,660.972151,387.545274,144.329057,252.254610,1105.933202,457.612672,57.203891,79.207708,917.731495,...,86.848961,4.629741,15.481880,56.572955,54.512745,800.861102,385.542364,66.295823,40.338724,406.962937
6036,1880.113889,470.208790,271.082568,92.726256,168.836321,862.877018,318.830160,40.738742,52.526274,666.034570,...,67.618101,3.413258,10.325325,41.167897,47.689492,620.295478,294.875972,51.832699,29.851082,322.005972
6037,654.944340,157.218963,96.651651,33.259711,61.588459,234.085809,124.227058,14.008366,15.147327,212.912878,...,19.484473,1.198106,4.278779,11.767266,10.985585,200.185810,83.823399,15.090288,8.001675,92.627905
6038,1223.766423,318.082570,181.852773,61.376827,117.841536,430.533050,231.945118,30.870166,27.681883,395.280092,...,41.942428,3.077609,8.548679,21.955715,30.724809,359.644195,161.975146,30.371656,17.349157,176.789357


There we go ! We now have for each user (rows), a prediction on how they would be interested in each item (columns). We should note that we haven't excluded the items they have already seen. For example if we look at the first line and column we can see that the recommendation score is very high because that user is similar to himself and has rated this movie 5/5 stars.

In [59]:
def sort_predictions_with_indices(items_predictions_arr):

    # Use numpy to get sorted indices for descending order
    sorted_indices = np.argsort(-items_predictions_arr, axis=1)

    # Use sorted indices to arrange the array in sorted order
    sorted_predictions = np.take_along_axis(items_predictions_arr, sorted_indices, axis=1)

    # Create DataFrames for the sorted values and their original indices
    df_sorted_predictions = pd.DataFrame(sorted_predictions)
    df_sorted_indices = pd.DataFrame(sorted_indices)

    return df_sorted_predictions, df_sorted_indices

items_predictions_arr = np.array(items_predictions)

df_sorted_predictions, df_sorted_indices = sort_predictions_with_indices(items_predictions_arr)

df_sorted_predictions


,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,1882.860602,1855.301237,1790.254094,1652.093987,1596.497786,1568.449101,1563.344098,1558.219104,1553.619590,1535.142613,...,0.041208,0.032136,0.031416,0.024809,0.012335,0.000000,0.000000,0.000000,0.000000,0.000000
1,2506.589496,2475.785398,2466.463039,2333.647947,2250.487425,2220.229448,2195.984854,2172.580868,2146.279037,2124.192110,...,0.066184,0.048777,0.044907,0.040715,0.040577,0.032411,0.019446,0.011901,0.006482,0.000000
2,2306.011405,2243.836149,2162.088009,2058.791026,1999.506718,1856.092282,1824.930713,1819.691081,1791.021257,1769.356117,...,0.055292,0.042376,0.041392,0.035147,0.030157,0.025010,0.022814,0.013797,0.000000,0.000000
3,2147.714497,2004.075622,1857.431001,1753.301183,1685.377716,1664.401598,1607.665969,1542.519690,1540.440382,1448.098270,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2549.883607,2015.748533,2012.700393,1972.028063,1957.609924,1947.425631,1924.752867,1910.941463,1865.589803,1836.328564,...,0.069478,0.058383,0.049302,0.042691,0.042121,0.039508,0.029523,0.026339,0.010799,0.003683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,3593.560215,3511.185261,3448.831443,3049.256702,3019.682144,3000.960330,2978.118579,2977.662368,2957.690755,2855.316488,...,0.194524,0.155457,0.153844,0.137552,0.096331,0.093887,0.092306,0.046837,0.031894,0.030769
6036,3077.796423,3068.439929,2930.099372,2667.365343,2655.282223,2580.176161,2552.008106,2548.105399,2480.163001,2478.817517,...,0.109393,0.091229,0.090533,0.083430,0.070080,0.063081,0.050058,0.035092,0.016686,0.000000
6037,920.690721,912.764751,906.064787,890.253554,807.263382,763.653884,762.578247,741.693741,729.235310,728.557486,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6038,1918.889559,1783.813300,1699.419839,1583.834515,1554.709411,1515.144780,1505.536303,1469.877650,1429.783222,1405.008258,...,0.031445,0.031049,0.026265,0.021458,0.020734,0.016688,0.010350,0.000000,0.000000,0.000000


In [60]:
df_sorted_indices

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,253,2651,1106,1848,1108,513,579,1120,1178,2557,...,3597,2989,696,633,128,3054,389,815,3406,658
1,2651,1106,253,1848,2374,1108,579,1120,575,106,...,1664,658,815,128,633,3152,3097,3054,122,1735
2,253,1106,2651,1108,1120,2374,1107,1848,1178,575,...,2989,658,3097,2116,3054,633,696,122,1652,1654
3,253,1106,1108,1120,1848,2374,575,2651,1148,466,...,3054,1735,1652,2245,1654,3097,3536,658,2989,3152
4,2651,593,579,253,2557,1106,1848,2374,309,287,...,815,3071,389,658,122,3064,633,1565,128,3054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,2651,253,1106,1108,2374,579,1120,1848,593,575,...,2638,774,3152,622,2989,633,3097,128,658,122
6036,2651,253,1106,579,593,1108,802,1848,513,1120,...,633,774,696,3152,1735,2989,3097,128,122,658
6037,253,1106,2651,1120,1108,1178,1107,1848,579,2374,...,1654,3414,1652,3054,122,658,633,622,3152,926
6038,253,1106,2651,1108,1120,802,1178,1107,593,579,...,622,3097,633,2638,696,658,122,1735,3054,2989


The reason the functions returns two separate data frames instead of only one containing the indices and the values at the same time is due to performance. The way I tried before that was by matching every value with it's tuple and then 
sorting it in order to keep the original indices, unfortunately this function was very slow due to the nature of the way Python code is executed. Using NumPy functions only allows the code to execute at runtime, thus being way fatser. Time and space complexity remains the same. The only difference here
is the speed of execution of our code.

With that, we finished building our simple user-based collaborative filtering recommander system ! Note that we could transform it in a item-based system easily, by transposing our rating matrix.

## 2. Exploiting a recommender system

Now that we have finished implementing a simple version of our recommending system, we will explore the different attacks that we can perform against it and how to mitigate those potential attacks.

### 2.1 Component-based poisoning

First we are going to cover the attacks that exploits the different components of our RS, from data parsing to rating and optimization.

#### 2.1.1 System degradation attack (Shilling)

The first attack that I will conduct against our RS is an attack that aims at decreasing the accuracy of our RS by inflating or deflating our ratings. To do this will use "shilling". Shilling is a type of attack that consists in crafting and injecting fake users profile into the system.

So our objective will be to degrade the system using fake users, those users will rate one specific movie with a bad rating in order to make him less recommended.

In [72]:
# As detailled in the dataset 'README' file
column_names = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

users_data = pd.read_csv('datasets/movies/users.dat', engine='python', delimiter='::', names=column_names)

users_data

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


Here are the diffrents informations about our users, they are described as follow in the README:

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"


We can now use those informations to generate random user informations



In [74]:
import random

def generate_random_users(n, index):
    # Possible age codes and their corresponding ranges
    age_codes = [1, 18, 25, 35, 45, 50, 56]
    
    # Possible occupation codes
    occupation_codes = range(21)  # 0 to 20

    users = []

    for i in range(n):
        user_id = index+1 + i
        gender = random.choice(['M', 'F'])
        age_code = random.choice(age_codes)
        occupation_code = random.choice(list(occupation_codes))
        zip_code = f"{random.randint(10000, 99999)}"
        
        user = f"{user_id}::{gender}::{age_code}::{occupation_code}::{zip_code}" + "\n"
        users.append(user)
    
    return users

generate_random_users(10, 100)

['101::M::1::6::54365\n',
 '102::M::18::7::96355\n',
 '103::M::1::20::37774\n',
 '104::M::50::20::56057\n',
 '105::M::18::5::96155\n',
 '106::F::1::2::34324\n',
 '107::M::35::9::90226\n',
 '108::F::18::9::63429\n',
 '109::F::56::5::29463\n',
 '110::F::56::3::94107\n']

This method will alow us to create $n$ number of randomly generated users. Now let's create a poisoned dataset

In [116]:
users_content : list
with open('datasets/movies/users.dat', 'r') as f:
    users_content = f.read().split('\n')
    
index_id = int(users_content[-2].split('::')[0])
fake_users = generate_random_users(604, index_id)  # 10 percent of the dataset

with open('datasets/movies/users_poisoned.dat', 'w') as f:
    f.close()
    #users_content.extend(fake_users)
   # f.writelines(fake_users)

#with open('datasets/movies/users_poisoned.dat', 'w') as f:
  

We now have a created fake users in the database (UserID 6041 to 6644). The next step to our plan is to create reviews for those newly added users.

In [120]:
TARGET_FILM = 253 # Interview with the Vampire (1994)

from time import time # to get unix timestamp
def create_rating(user_id, film_id, rating):
    return f"{user_id}::{film_id}::{rating}::{int(time())}"

create_rating(1, 253, 1)


'1::253::1::1709737930'

In [145]:
START_ID = 6041
END_ID = 6644
from io import StringIO
fake_ratings = list()
poisoned_ratings_df = ratings_data

for id in range(START_ID, END_ID+1): # iterating through every fake users id
    fake_ratings.append(create_rating(id, TARGET_FILM, 1))

column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']
#creating a DataFrame for our new ratings
fake_ratings_df = pd.read_csv(StringIO("\n".join(fake_ratings)), delimiter="::", engine='python', names=column_names)
fake_ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,6041,253,1,1709739140
1,6042,253,1,1709739140
2,6043,253,1,1709739140
3,6044,253,1,1709739140
4,6045,253,1,1709739140


In [146]:
poisoned_ratings_df = pd.concat([poisoned_ratings_df, fake_ratings_df], ignore_index=True)

poisoned_ratings_df

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000808,6640,253,1,1709739140
1000809,6641,253,1,1709739140
1000810,6642,253,1,1709739140
1000811,6643,253,1,1709739140


As we can see w've successfully added our fake ratings to the database. We now need to save it.

In [154]:
concatenated_series = poisoned_ratings_df.astype(str).apply('::'.join, axis=1)
single_column_df = concatenated_series.to_frame()

single_column_df.to_csv('datasets/movies/ratings_poisoned.dat', header=False, index=False, sep='\t')


Note: This weird way of exporting is due to the limitation of Pandas on multi-character delimiters.